# UTILITIES

## Get Dummies

Just get the dummies

In [ ]:
titanic_features = ['sex', 'age', 'sibsp', 'parch', 'fare']
X = pd.get_dummies(titanic[titanic_features])

Keep the previous data `AND REMOVE PREVOUS COLUMNS`

In [ ]:
categorical_features = [
 'animal_type',
 'intake_condition',
 'intake_type',
 'sex_upon_intake',
 'sex_upon_outcome',
 'outcome_type'
]
data_dummies = pd.get_dummies(original_data, columns=categorical_features, drop_first=True)

## Standarize: `FIRST SPLIT THEN STD`

In [ ]:
from sklearn.preprocessing import StandardScaler


X_train_norm = StandardScaler().fit(X_train).transform(X_train)
X_test_norm = StandardScaler().fit(X_test).transform(X_test)
print(f"MEAN Training after Norm: {X_train_norm.mean(axis=0)}")
print(f"STD Training after Norm: {X_train_norm.std(axis=0)}")

In [ ]:
print(f"BEFORE")
print(f"mean {X[to_std_columns].mean(axis=0)}")
print(f"std {X[to_std_columns].std(axis=0)}\n\n")

to_std_columns = ['area', 'perimeter', 'compactness', 'lengthOfKernel',
       'widthOfKernel', 'asymmetryCoefficient', 'lengthOfKernelGroove']
# to_std_columns = X.columns

X_std = X.copy()
X_std[to_std_columns] = (X[to_std_columns] - X[to_std_columns].mean(axis=0)) / X[to_std_columns].std(axis=0)

print(f"AFTER")
print(f"mean {X_std[to_std_columns].mean(axis=0)}")
print(f"std {X_std[to_std_columns].std(axis=0)}")

### Consider adding a column of 1s to add biases in some models

In [ ]:
X_train_norm_const = sm.add_constant(X_train_norm)
X_test_norm_const = sm.add_constant(X_test_norm)

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_df = toyota_dummies_df.drop(columns="Price")
y_df = toyota_dummies_df[["Price"]]

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)
print(f"Training with {len(X_train)} samples")
print(f"Testing with {len(X_test)} samples")

In [ ]:
def train_test_spli(data, X_remove, y_col, proportion: float = 0.8, seed: int = 42):
    random.seed(seed)
    N = len(data)
    indices = list(range(N))
    random.shuffle(indices)
    split = int(N*proportion)

    X = data.drop(columns=X_remove)
    y = data[y_col]
    X_train, y_train = X.iloc[:split], y.iloc[:split]
    X_test , y_test  = X.iloc[split:], y.iloc[split:]
    
    print(f"Training with {len(X_train)} samples")
    print(f"Testing with {len(X_test)} samples")
    return X_train, y_train, X_test , y_test

X_train, y_train, X_test , y_test = train_test_spli(data_dummies, X_remove=['adopted', 'outcome_type'], y_col="adopted")

# Metrics

## Confusion Matrix + F_score

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true, y_pred)
# normalize
conf_matrix = conf_matrix / len(y_true)
print(conf_matrix)

In [ ]:
def confusion_matrix(y_pred, y_true, verbose: bool = True, plot: bool = False):
    tp = np.sum((y_pred == 1) & (y_true == 1))  # True Positives
    tn = np.sum((y_pred == 0) & (y_true == 0))  # True Negatives
    fp = np.sum((y_pred == 1) & (y_true == 0))  # False Positives
    fn = np.sum((y_pred == 0) & (y_true == 1))  # False Negatives

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    
    if verbose:
        print(f"{tp = :6d} | {fp = :6d}")
        print(f"----------- |  -----------")
        print(f"{fn = :6d} | {tn = :6d}")
        print(f"{precision = :.4}")
        print(f"{recall = :.4}")
        print(f"{f_score = :.4}")
        
    if plot:
        confusion_matrix = np.array([[tp, fn],  
                                    [fp, tn]]) 

        confusion_matrix = pd.DataFrame(confusion_matrix, 
                                    index=["Actual Positive", "Actual Negative"], 
                                    columns=["Predicted Positive", "Predicted Negative"])

        plt.figure(figsize=(8, 6))
        sn.heatmap(confusion_matrix, annot=True, fmt="d", cmap='YlGnBu', cbar=False)
        plt.title("Confusion Matrix")
        plt.ylabel("Actual")
        plt.xlabel("Predicted")
        plt.tight_layout()
        plt.show()


In [ ]:
f_scores = []
precisions = []
recalls = []
thresholds = [i*0.1 for i in range(11)]
# thresholds = np.linspace(0, 1, 100)
for threshold in thresholds:
    pred = distribution[:,1] > threshold
    f_score, precision, recall = confusion_matrix(pred, y_test, verbose=False, plot=False)
    f_scores += [f_score]
    precisions += [precision]
    recalls += [recall]
    
plt.figure(figsize=(8,6))

plt.plot(thresholds, f_scores, marker="o", label="f_scores")
plt.plot(thresholds, precisions, marker="o", label="precisions")
plt.plot(thresholds, recalls, marker="o", label="recalls")
# plt.plot(thresholds, f_scores, marker="", label="f_scores")
# plt.plot(thresholds, precisions, marker="", label="precisions")
# plt.plot(thresholds, recalls, marker="", label="recalls")

plt.xlabel('Thresholds')
plt.ylabel('Score')
plt.title(f'Score vs Thresholds')
# plt.ylim(0.9, 1.0)
plt.xlim(thresholds[0] - 0.1, thresholds[-1] + .1)
plt.tight_layout()
plt.legend()


### Accuracy

In [ ]:
def accuracy_score(y_test, y_test_pred):
    N = len(y_test)
    correct = np.sum(y_test == y_test_pred)
    return correct / N

In [ ]:
y_test_pred = array([1, 3, 5, ..., 1, 4, 2], dtype=int64)

### R^2 Score

R2 score metric (wtf is this)

- The R² score on test set is 0.812, which means that 81.2% of the variance in the target variable (Price) can be explained by the features in the model. 

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)
print("R2 score on test dataset: ", r2)

## Dimension reduction

- TSNE
- PCA

In [ ]:
X_reduced_tsne = TSNE(n_components=2, init='random', learning_rate='auto', random_state=0).fit_transform(X)

print("The features of the first sample are: %s" % X_reduced_tsne[0])

In [ ]:
X_reduced_pca = PCA(n_components=2).fit(X).transform(X)

print("The features of the first sample are: %s" % X_reduced_pca[0])

### Plot

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(7,3), sharey=True)

# Cluster the data in 3 groups
labels = KMeans(n_clusters=3, random_state=0).fit_predict(X)

# Plot the data reduced in 2d space with t-SNE
axs[0].scatter(X_reduced_tsne[:,0], X_reduced_tsne[:,1], c=labels, alpha=0.6)
axs[0].set_title("t-SNE")

# Plot the data reduced in 2d space with PCA
axs[1].scatter(X_reduced_pca[:,0], X_reduced_pca[:,1], c=labels, alpha=0.6)
axs[1].set_title("PCA")

# Regression 1

you do not need a train/test split if you want to get the propensy score. Train and apply the model on the entire dataset. If you're wondering why this is the right thing to do in this situation, recall that the propensity score model is not used in order to make predictions about unseen data. Its sole purpose is to balance the dataset across treatment groups.
(See p. 74 of Rosenbaum's book for an explanation why slight overfitting is even good for propensity scores.

### `STANDARIZE`

In [ ]:
def standarize_column(names: list[str], df: pd.DataFrame):
    for name in names:
        df[name] = (df[name] - df[name].mean())/df[name].std()
        
standarize_column(["age", "creatinine_phosphokinase", "ejection_fraction", "platelets", "serum_creatinine", "serum_sodium"], df)

### Part 1 Linear regression: Modelling time spent at the hospital

- We will perform a regression analysis to model the number of days spent at the hospital, among the population of patients.


- To get started with our model, we need two components:

   1. The equation describing the model
   2. The data
   
   
- Equations are specified using patsy formula syntax. Important operators are:
    1. `~` : Separates the left-hand side and right-hand side of a formula.
    2. `+` : Creates a union of terms that are included in the model.
    3. `:` : Interaction term.
    3. `*` : `a * b` is short-hand for `a + b + a:b`, and is useful for the common case of wanting to include all interactions between a set of variables.
    
    
- Intercepts are added by default.


- Categorical variables can be included directly by adding a term C(a). More on that soon!


- For (2), we can conveniently use pandas dataframe.

### An example

- Let's start with an example from our dataset. We are interested in two predictors: diabetes and high blood pressure. These are the two predictors that we want to use to fit the outcome, the number of days spent at the hospital, using a linear regression.

- A model that achieves this is formulated as:
        time ~ C(diabetes) + C(high_blood_pressure)
        
- We can create this model using smf.ols().

- OLS stands for ordinary least squares linear regression.

- The two components: the formula and the data are stated explicitly.

- The terms in the formula are columns in pandas dataframe. Easy!

In [ ]:
import statsmodels.api as sm

### Ordinal least Squares 

##### Add a column of ones so it can get a bias in the model

In [ ]:
X_train_norm_const = sm.add_constant(X_train_norm)
X_test_norm_const = sm.add_constant(X_test_norm)

In [ ]:
logistic_regression_model = sm.OLS(y_train, X_train_norm).fit()
print(logistic_regression_model.summary())

In [ ]:
mod = smf.ols(formula='time ~ C(diabetes) + C(high_blood_pressure)', data=df)
mod = smf.ols(formula='time ~ C(high_blood_pressure) * C(DEATH_EVENT,  Treatment(reference=0)) + C(diabetes)', data=df)

res = mod.fit()
print(res.summary())
# Confident intervals
res.conf_int()

### Full process keeping the names

In [ ]:
scaler = StandardScaler()
X_train_ext_norm = pd.DataFrame(scaler.fit_transform(X_train_ext), columns=X_train_ext.columns, index=X_train_ext.index)
X_train_ext_norm_const = sm.add_constant(X_train_ext_norm)

lr_model = sm.OLS(y_train, X_train_ext_norm_const).fit()
print(lr_model.summary())

**Comment:** In the first model, `high_blood_pressure` is associated with an additive coefficient of around -25. Thus, in the model, whenever a patient has high blood pressure we deduce -25 days out of the prediction.
In the second model, `high_blood_pressure` is associated with an multiplicative coefficient of around -0.22. This means that, in the model, whenever a patient has high blood pressure we multiply his or her outcome by $e^{-0.22} \simeq 0.80$. 

### Logistic regression

This coeficients show how the `log-odds` will change when unitary change of the variable happens
- To go from p to odds: $\text{odds} = \frac{p}{1 - p}$
- To go from p to log-odds: $\text{log-odds} = \ln\left(\frac{p}{1 - p}\right)$
- To go from log-odds to odds: $\text{odds} = e^{\text{log-odds}}$
- To go from log-odds to p: $p = \frac{1}{1 + e^{-\text{log-odds}}}$
- To go from odds to log-odds: $\text{log-odds} = \ln(\text{odds})$
- To go from odds to p: $p = \frac{\text{odds}}{1 + \text{odds}}$

In [ ]:
def p_to_log_odd(p):
    return  np.log(p/(1-p))
def log_odd_to_p(log_odd):
    return 1/(1+np.exp(-log_odd))

p = 0.1
original_log_odd = p_to_log_odd(p)
new_log_odd = original_log_odd + 0.66

new_p = log_odd_to_p(new_log_odd)
print(f"{p = }")
print(f"{original_log_odd = }")
print(f"{new_log_odd = }")
print(f"{new_p = }, increased by {p-new_p} \n")

In [ ]:
mod = smf.logit(formula='DEATH_EVENT ~  age + creatinine_phosphokinase + ejection_fraction + \
                        platelets + serum_creatinine + serum_sodium + \
                        C(diabetes) + C(high_blood_pressure) +\
                        C(sex) + C(anaemia) + C(smoking) + C(high_blood_pressure)', data=df)
res = mod.fit()
print(res.summary())

##### A lot of useful information is provided by default.

`Coef` is the value assigned to each `variable` + the `bias`

- The dependent variable : time (number of days at the hospital)
- Method: The type of model that was fitted (OLS)
- Nb observations: The number of datapoints (299 patients)
- R2: The fraction of explained variance
- A list of predictors
- For each predictor: coefficient, standard error of the coefficients, p-value, 95% confidence intervals. 
  - We can see that one is a significant predictor if p < 0.5 (or very small numbers)
- Warnings if there are numerical issues (hopefully not!)

### Predict / Propensity Score

In [ ]:
df["propensity_score"] = res.predict()

### Get the values

In [ ]:
variables = lr_model.params.index
coefficients = lr_model.params.values

for coefficient, name in zip(coefficients, variables):
    # if name != "high_blood_pressure": continue
    # print(f"A change in {name} of 1 unit, increases your dead probability by ~{coefficient: 0.4f}")
    print(f"{name: <15}: {coefficient: 0.4f}")

In [ ]:
variables = res.params.index
coefficients = res.params.values

for coefficient, name in zip(coefficients, variables):
    # if name != "high_blood_pressure": continue
    print(f"A change in {name} of 1 unit, increases your dead probability by ~{coefficient: 0.4f}")


In [ ]:
# feature names
variables = res.params.index

# quantifying uncertainty!

# coefficients
coefficients = res.params.values

# p-values
p_values = res.pvalues

# standard errors
standard_errors = res.bse.values

l1, l2, l3, l4 = zip(*sorted(zip(coefficients[1:], variables[1:], standard_errors[1:], p_values[1:])))
# fancy plotting 
plt.errorbar(l1, np.array(range(len(l1))), xerr= 2*np.array(l3), linewidth = 1,
             linestyle = 'none',marker = 'o',markersize= 3,
             markerfacecolor = 'black',markeredgecolor = 'black', capsize= 5)

plt.vlines(0,0, len(l1), linestyle = '--')

plt.yticks(range(len(l2)),l2)

## Significancy Vriables (p < 0.05)

GET THE NAME OF THE COLUMS

In [ ]:
p_values = lr_model.pvalues

significant_features = p_values[p_values < 0.05].index

# IF a constant column is added
significant_features = significant_features.drop('const')

print("Significant features: ", significant_features)

## Matching propensity

In [ ]:
df["propensity_score"] = res.predict()
treated = df[df["treat"] == 1]
control = df[df["treat"] == 0]

def similarity(p_1, p_2):
    return 1-np.abs(p_1 - p_2)

# THE HIGHER THE EASIER FOR THEM TO MATCH
def get_similarity(control_row, treatment_row):
    return(
        1
        - np.abs(control_row["propensity_score"] - treatment_row["propensity_score"])
        - int(control_row["black"]  !=  treatment_row["black"])
        - int(control_row["hispan"]  !=  treatment_row["hispan"])
    )

In [ ]:
# Create an empty undirected graph
G = nx.Graph()

# Loop through all the pairs of instances
for control_id, control_row in control_df.iterrows():
    for treatment_id, treatment_row in treatment_df.iterrows():

        # Calculate the similarity 
        similarity = similarity(
            # control_row['Propensity_score'], treatment_row['Propensity_score']
            control_row, treatment_row
        )

        # Add an edge between the two instances weighted by the similarity between them
        G.add_weighted_edges_from([(control_id, treatment_id, similarity)])

# Generate and return the maximum weight matching on the generated graph
matching = nx.max_weight_matching(G)

matched = [i[0] for i in list(matching)] + [i[1] for i in list(matching)]

balanced_df_1 = lalonde_data.iloc[matched]

# Regression 2

### Logistic regression

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, auc, roc_curve

In [ ]:
logistic = LogisticRegression(solver='lbfgs')
logistic.fit(X.values, y)

target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

### Evaluation

In [ ]:
men = np.array[[...],[...]]
pred = logistic.predict([men])
distribution = logistic.predict_proba([men])
error = mean_squared_error(y, pred)

In [ ]:
mean_squared_error(y_test["sold_within_3_months"].values, pred)

### Cross Validation

In [ ]:
y_pred = cross_val_predict(logistic, X, y, cv=10, method="predict_proba")
precision = cross_val_score(logistic, X, y, cv=10, scoring="precision").mean()
recall = cross_val_score(logistic, X, y, cv=10, scoring="recall").mean()

### ROC Curve

In [ ]:
# Predict the probabilities with a cross validationn
y_pred = cross_val_predict(logistic, X, y, cv=10, method="predict_proba")
# Compute the False Positive Rate and True Positive Rate
fpr, tpr, _ = roc_curve(y, y_pred[:, 1])
# Compute the area under the fpt-tpf curve
auc_score = auc(fpr, tpr)

### Threshold optimization (Linear and binary search)

In [ ]:
# TODO
distribution = logistic.predict_proba(X_test_norm_const)
thresholds = [i*0.001 for i in range(1001)]

best_f_score = -float("inf")
best_thres = None
for thres in thresholds:
    pred = distribution[:,0] < thres
    score =  f_score(y_test["sold_within_3_months"].values, pred)
    print(f"  · {score = :0.4}")
    if score > best_f_score:
        best_f_score = score
        best_thres = thres
        
print(f"Best Threshold {best_thres} with {best_f_score = :0.4f}")

In [ ]:
distribution = logistic.predict_proba(X_test_norm_const)

def binary_threshold_search(distribution, max_iter: int = 10):
    ini, end = 0, 1
    mid = (ini + end)/2
    
    for i in range(max_iter):
        pred = distribution[:,0] < end
        end_score =  f_score(y_test["sold_within_3_months"].values, pred)
        pred = distribution[:,0] < mid
        mid_score =  f_score(y_test["sold_within_3_months"].values, pred)
        if mid_score > end_score:
            end = mid
        else: 
            ini = mid
        
        mid = (ini + end)/2

    return mid, mid_score
        
best_thres, best_f_score = binary_threshold_search(distribution)
print(f"Best Threshold {best_thres} with {best_f_score = :0.4f}")

# K-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(3) #k
model.fit(X, y_moons)
Z = model.predict([X[0]])
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

# K-MEANS

In [ ]:
from sklearn.cluster import KMeans, DBSCAN

kmean = KMeans(n_clusters=n_clusters, random_state=42).fit(X)
# Plot the data by using the labels as color
ax.scatter(X[:,0], X[:,1], c=kmean.labels_, alpha=0.6)

for c in kmean.cluster_centers_:
    ax.scatter(c[0], c[1], marker="+", color="red")

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(4,4), sharey=True)

# Plot the clusters with K = 3
labels = KMeans(n_clusters=3, random_state=0).fit_predict(X)
axs.scatter(X[:,0], X[:,1], c=labels, alpha=0.6)

### Compare k

In [ ]:
MIN_CLUSTERS = 2
MAX_CLUSTERS = 10

# Compute number of row and columns
COLUMNS = 3
ROWS = math.ceil((MAX_CLUSTERS-MIN_CLUSTERS)/COLUMNS)
fig, axs = plt.subplots(ROWS, COLUMNS, figsize=(10,8), sharey=True, sharex=True)

# Plot the clusters
for n_clusters in range(MIN_CLUSTERS, MAX_CLUSTERS+1):
    current_column = (n_clusters-MIN_CLUSTERS)%COLUMNS
    current_row = (n_clusters-MIN_CLUSTERS)//COLUMNS
    # Get the axis where to add the plot
    ax = axs[current_row, current_column]
    # Cluster the data with the current number of clusters
    kmean = KMeans(n_clusters=n_clusters, random_state=42).fit(X)
    # Plot the data by using the labels as color
    ax.scatter(X[:,0], X[:,1], c=kmean.labels_, alpha=0.6)
    ax.set_title("%s clusters"%n_clusters)
    ax.set_xlabel("Feature 1")
    ax.set_ylabel("Feature 2")
    # Plot the centroids
    for c in kmean.cluster_centers_:
        ax.scatter(c[0], c[1], marker="+", color="red")

plt.tight_layout()

## Silhouette and Elbow 
Silhouette suggests that using n clusters is a fair tradeoff between the number of groups and their separation. The elbow method shows how the SSE reduction is less significant with more than n clusters.

### Silhouette

- `silhouette_score(X, labels)`

In [ ]:
from sklearn.metrics import silhouette_score

silhouettes = []

# Try multiple k
for k in range(2, 11):
    # Cluster the data and assigne the labels
    labels = KMeans(n_clusters=k, random_state=10).fit_predict(X)
    # Get the Silhouette score
    score = silhouette_score(X, labels)
    silhouettes.append({"k": k, "score": score})
    
# Convert to dataframe
silhouettes = pd.DataFrame(silhouettes)

# Plot the data
plt.plot(silhouettes.k, silhouettes.score)
plt.xlabel("K")
plt.ylabel("Silhouette score")

### Elbow method

- `kmeans.inertia_`

In [ ]:
def plot_sse(X, start=2, end=11):
    sse = []
    for k in range(start, end):
        # Assign the labels to the clusters
        kmeans = KMeans(n_clusters=k, random_state=10).fit(X)
        sse.append({"k": k, "sse": kmeans.inertia_})

    sse = pd.DataFrame(sse)
    # Plot the data
    plt.plot(sse.k, sse.sse)
    plt.xlabel("K")
    plt.ylabel("Sum of Squared Errors")
    
plot_sse(X)

# DBSCAN

In [ ]:
ax = axs[current_row, current_column]

labels = DBSCAN(eps=eps).fit_predict(X_moons)

ax.scatter(X_moons[:,0], X_moons[:,1], c=labels, alpha=0.6)
ax.set_title("eps = {:.3f}".format(eps))    

### Compare epsilon

In [ ]:
# Create a list of eps
eps_list = np.linspace(0.05, 0.15, 14)

# Compute number of row and columns
COLUMNS = 7
ROWS = math.ceil(len(eps_list)/COLUMNS)

fig, axs = plt.subplots(ROWS, COLUMNS, figsize=(12, 4), sharey=True, sharex=True)

for i in range(0, len(eps_list)):
    eps = eps_list[i]
    
    current_column = i%COLUMNS
    current_row = i//COLUMNS
    
    ax = axs[current_row, current_column]
    labels = DBSCAN(eps=eps).fit_predict(X_moons)
    ax.scatter(X_moons[:,0], X_moons[:,1], c=labels, alpha=0.6)
    ax.set_title("eps = {:.3f}".format(eps))
    
plt.tight_layout()

# TREES

### Train

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decision_trees = DecisionTreeClassifier(random_state=42)
decision_trees = DecisionTreeClassifier(max_depth=8, random_state=42)

decision_trees.fit(X_train, y_train)

### Test

In [ ]:
y_test_pred = decision_trees.predict(X_test)

acc = accuracy_fn(y_test, y_test_pred)
depth = decision_trees.get_depth()

print(f"Accuracy: {acc: 0.4f}, Depth: {depth}")   

### Visualize

In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize=(20, 10))
plot_tree(decision_trees, filled=True)
plt.show()

In [ ]:
tree_clf = sklearn.tree.DecisionTreeClassifier(max_depth=1, random_state=42)
tree_clf_trained = tree_clf.fit(X_train, y_train)

plt.figure(figsize=(20, 10))  
sklearn.tree.plot_tree(tree_clf_trained, feature_names=X_train.columns, filled=True, class_names=['Not sold', 'Sold'], fontsize=30)  
plt.show()

### Compare depths

In [ ]:
depths = list(range(1, max_deph + 1))
acc_hist = []
for depth in depths:
    decision_trees = DecisionTreeClassifier(max_depth=depth, random_state=42)
    decision_trees.fit(X_train, y_train)
    y_test_pred_rf = decision_trees.predict(X_test)

    acc = accuracy_score(y_test["sold_within_3_months"].values, y_test_pred_rf)
    acc_hist += [acc]
    print(f"- Accuracy: {acc: 0.4f}, Depth: {depth}")   
    
    
plt.figure(figsize=(8,6))
plt.plot(depths, acc_hist, marker="o")

plt.xlabel('Max depth')
plt.ylabel('Accuracy')
plt.title(f'Accuracy vs max depth')
plt.ylim(0.9, 1.0)
plt.xlim(depths[0] - 1, depths[-1] + 1)
plt.tight_layout()